# Création du modèle pour la génération de Texte


Importation des packages :


In [1]:
# Installer tensorflow 2.0
# Après l'installation redémarrer l'environnement d'exécution
# en faisant attention à ne pas réexécuter cette cellule.
!pip install --upgrade tensorflow
## cliquer sur Restart Runtime pour utiliser plus tard tensorflow, pour que ça fonctionne

Requirement already up-to-date: tensorflow in /usr/local/lib/python3.6/dist-packages (2.1.0)


In [3]:
import pickle
import pandas as pd
from random import randint
import re
import unicodedata 
import time
import pandas as pd
import numpy as np
import random
import nltk
nltk.download('stopwords')
import collections
import csv
import pickle
from pandas import DataFrame
from sklearn.feature_extraction.text import TfidfVectorizer # ou CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from __future__ import absolute_import, division, print_function, unicode_literals
import os
import time
import re
import unicodedata
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

#pour importer les fichiers : 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authentification Google
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import tensorflow as tf

print(tf.__version__)

2.1.0


Importation des fichiers : 

In [0]:
downloaded = drive.CreateFile({'id': '1vJAynhWpDwu7kJtJjOLz8o7QBNEud6bO'})
downloaded.GetContentFile('dtm_question.pk')

dtm_question = pd.read_pickle('dtm_question.pk')

In [0]:
downloaded = drive.CreateFile({'id': '1RLUGruwvQ0j68Ug1kf1cV1k9g9H5BjRf'})
downloaded.GetContentFile('dtm_reponse.pk')

dtm_reponse = pd.read_pickle('dtm_reponse.pk')

In [0]:
downloaded = drive.CreateFile({'id': '1Py73_AkF3WjX9oAxqaso02T2A5dEp7Rc'})
downloaded.GetContentFile('dtm_thematiques.pk')

dtm_thematiques = pd.read_pickle('dtm_thematiques.pk')

In [0]:
downloaded = drive.CreateFile({'id': '1uKGzZMfI7BlWF3evEmifLUzMfIKYJrYe'})
downloaded.GetContentFile('dataQR.csv')

data = pd.read_csv('dataQR.csv', sep=';')

In [0]:
downloaded = drive.CreateFile({'id': '1JstvTue5GefBZD85nZkVE2ojIadeudfu'})
downloaded.GetContentFile('dataQR_normalize.csv')

data_norm = pd.read_csv('dataQR_normalize.csv', sep=';')

In [0]:
downloaded = drive.CreateFile({'id': '1eLnvXsKV4Pb2h5oqLAp3fPe42JGxB52B'})
downloaded.GetContentFile('OpenSubtitles.txt')

sous_titres = []
with open("OpenSubtitles.txt", "r", encoding='utf-8') as f:
    for line in f.readlines():
        sous_titres.append(line.strip())

### Aperçu du corpus 

In [10]:
sous_titres[:5]

['La Dre Petersen est prêt à vous recevoir.',
 "Désolée. ll faut que j'y aille.",
 "J'avais un jeu parfait. Je vous aurais écrasées.",
 '- Harry va vous conduire.',
 'Surveillez-la attentivement.']

In [11]:
len(sous_titres)

276210

On conserve les 9000 premiers sous titres afin de réduire le temps de calcul.

In [0]:
sous_titres = sous_titres[:9000]

On concatène en une seule chaine de caractère toutes les phrases présentes dans 'sous_titres'.

In [0]:
concat_sous_titres = sous_titres[0]
for phrase in sous_titres[1:] :
  concat_sous_titres = '\n'.join([concat_sous_titres, phrase])


In [14]:
concat_sous_titres

'La Dre Petersen est prêt à vous recevoir.\nDésolée. ll faut que j\'y aille.\nJ\'avais un jeu parfait. Je vous aurais écrasées.\n- Harry va vous conduire.\nSurveillez-la attentivement.\nNe la quittez pas des yeux.\n- Bien. - Vous semblez un peu bilieux.\n- C\'est l\'éclairage.\nJe m\'inquiète pour vous.\nÇa va aller.\nDoit-on aller au bureau de la Dre Petersen ?\nOn ne peut pas aller discuter dans un endroit tranquille ?\nJ\'aimerais bien, si j\'avais le temps.\nVraiment ?\nEntrez.\nVous avez gâché une partie de cartes passionnante, Dre Petersen. Vous pouvez disposer, Harry.\nJe vais attendre dehors.\nJ\'espère que vous allez mieux, Mary. - Pas du tout. - Ça va venir.\nJe trouve tout ça complètement idiot.\nTout ça ?\nLa psychanalyse.\nÇa me barbe complètement.\nS\'allonger sur le divan comme une imbécile, tout raconter.\nVous n\'imaginez pas parvenir à quelque chose en m\'écoutant parler de mon enfance débile.\nMes patients me trouvent toujours prodigieusement agaçante à nos premières

Pré traitement des textes : 

In [15]:
concat_sous_titres = concat_sous_titres.replace('.','')
concat_sous_titres = concat_sous_titres.replace(" '","'")
concat_sous_titres = concat_sous_titres.replace('\n',' ')
concat_sous_titres = concat_sous_titres.replace('"',' ')
concat_sous_titres = concat_sous_titres.replace('-',' ')
concat_sous_titres = concat_sous_titres.replace('  ',' ')
concat_sous_titres = concat_sous_titres.lower()
concat_sous_titres

"la dre petersen est prêt à vous recevoir désolée ll faut que j'y aille j'avais un jeu parfait je vous aurais écrasées  harry va vous conduire surveillez la attentivement ne la quittez pas des yeux  bien  vous semblez un peu bilieux  c'est l'éclairage je m'inquiète pour vous ça va aller doit on aller au bureau de la dre petersen ? on ne peut pas aller discuter dans un endroit tranquille ? j'aimerais bien, si j'avais le temps vraiment ? entrez vous avez gâché une partie de cartes passionnante, dre petersen vous pouvez disposer, harry je vais attendre dehors j'espère que vous allez mieux, mary  pas du tout  ça va venir je trouve tout ça complètement idiot tout ça ? la psychanalyse ça me barbe complètement s'allonger sur le divan comme une imbécile, tout raconter vous n'imaginez pas parvenir à quelque chose en m'écoutant parler de mon enfance débile mes patients me trouvent toujours prodigieusement agaçante à nos premières rencontres je vois c'est mon subconscient qui s'énerve il ne veut 

In [0]:
#on garde uniquement les caractères alphanumériques : 
#on retire tous les accents 
concat_sous_titres = unicodedata.normalize('NFD', concat_sous_titres).encode('ascii', 'ignore')
concat_sous_titres = re.sub('[^a-z_\']', ' ', str(concat_sous_titres))
concat_sous_titres = concat_sous_titres[2::]

In [0]:
with open('concat_sous_titres.pk', 'wb') as fin:
    pickle.dump(concat_sous_titres, fin)

### Unicité des phrases présentes dans le corpus

On récupère chaque mot dans une liste en les séparant en fonction des espaces présents. On garde les éléments de manière unique, en les triant par ordre alphabétique.

In [18]:
vocab = sorted(np.unique(concat_sous_titres.split(' ')))
print ('{} unique words'.format(len(vocab)))

6553 unique words


In [0]:
with open('vocab.pk', 'wb') as fin:
    pickle.dump(vocab, fin)

### Sous titres en représentation numérique

On associé chaque mot des sous titres à un entier différent.

In [20]:
liste_concat_sous_titres = concat_sous_titres.split(' ')
liste_concat_sous_titres[0:4]

['la', 'dre', 'petersen', 'est']

In [0]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in liste_concat_sous_titres])

In [22]:
text_as_int

array([3431, 1944, 4479, ..., 1567, 3219,    0])

In [23]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  ''  :   0,
  'a' :   1,
  'abandonne':   2,
  'abandonner':   3,
  'abats':   4,
  'abattre':   5,
  'abattu':   6,
  'abattus':   7,
  'abbott':   8,
  'abdication':   9,
  'abner':  10,
  'abordez':  11,
  'abri':  12,
  'absente':  13,
  'absolu':  14,
  'absolument':  15,
  'absurdes':  16,
  'abuse':  17,
  'accelere':  18,
  'accelerer':  19,
  ...
}


### Affichez la représentation numérique des 13 premières phrases


In [24]:
print ('{} ---- characters mapped to int ---- > {}'.format(repr(concat_sous_titres[:13]), text_as_int[:13]))

'la dre peters' ---- characters mapped to int ---- > [3431 1944 4479 2207 4741    1 6503 5033 1737 3523 2365 4911 3087]


## Partie Prédictions de réponses
Si on passe une phrase comme question, quelle sera la réponse la plus probable ? Entrainement du modèle pour prédire la réponse qui suit.

In [25]:
# Longueur de la phrase maximale qu'on pourra obtenir avec un certain nombre de mots passés en argument
seq_length = 100
examples_per_epoch = len(liste_concat_sous_titres)//(seq_length+1)

# Création d'exemples d'entrainements
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

la
dre
petersen
est
pret


## Création de batchs

La méthode par lots nous permet de convertir facilement ces caractères individuels en séquences de la taille souhaitée

In [26]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(' '.join(idx2char[item.numpy()])))

"la dre petersen est pret a vous recevoir desolee ll faut que j'y aille j'avais un jeu parfait je vous aurais ecrasees  harry va vous conduire surveillez la attentivement ne la quittez pas des yeux  bien  vous semblez un peu bilieux  c'est l'eclairage je m'inquiete pour vous ca va aller doit on aller au bureau de la dre petersen   on ne peut pas aller discuter dans un endroit tranquille   j'aimerais bien  si j'avais le temps vraiment   entrez vous avez gache une partie de cartes passionnante  dre petersen vous pouvez"
"disposer  harry je vais attendre dehors j'espere que vous allez mieux  mary  pas du tout  ca va venir je trouve tout ca completement idiot tout ca   la psychanalyse ca me barbe completement s'allonger sur le divan comme une imbecile  tout raconter vous n'imaginez pas parvenir a quelque chose en m'ecoutant parler de mon enfance debile mes patients me trouvent toujours prodigieusement agacante a nos premieres rencontres je vois c'est mon subconscient qui s'enerve il ne veu

Appliquer une fonction à tous les batchs

In [0]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [28]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(' '.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(' '.join(idx2char[target_example.numpy()])))

Input data:  "la dre petersen est pret a vous recevoir desolee ll faut que j'y aille j'avais un jeu parfait je vous aurais ecrasees  harry va vous conduire surveillez la attentivement ne la quittez pas des yeux  bien  vous semblez un peu bilieux  c'est l'eclairage je m'inquiete pour vous ca va aller doit on aller au bureau de la dre petersen   on ne peut pas aller discuter dans un endroit tranquille   j'aimerais bien  si j'avais le temps vraiment   entrez vous avez gache une partie de cartes passionnante  dre petersen vous"
Target data: "dre petersen est pret a vous recevoir desolee ll faut que j'y aille j'avais un jeu parfait je vous aurais ecrasees  harry va vous conduire surveillez la attentivement ne la quittez pas des yeux  bien  vous semblez un peu bilieux  c'est l'eclairage je m'inquiete pour vous ca va aller doit on aller au bureau de la dre petersen   on ne peut pas aller discuter dans un endroit tranquille   j'aimerais bien  si j'avais le temps vraiment   entrez vous avez gac

In [29]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 3431 ('la')
  expected output: 1944 ('dre')
Step    1
  input: 1944 ('dre')
  expected output: 4479 ('petersen')
Step    2
  input: 4479 ('petersen')
  expected output: 2207 ('est')
Step    3
  input: 2207 ('est')
  expected output: 4741 ('pret')
Step    4
  input: 4741 ('pret')
  expected output: 1 ('a')


## Création de lots d'entrainements
We used tf.data to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [30]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

## Construction du modèle

In [0]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [0]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [0]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

## Test du modèle

In [34]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 6553) # (batch_size, sequence_length, vocab_size)


In [35]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           1677568   
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 6553)          6716825   
Total params: 12,332,697
Trainable params: 12,332,697
Non-trainable params: 0
_________________________________________________________________


In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [37]:
sampled_indices

array([2146,   37, 6519, 5653, 2277,  981, 1365, 5730, 4875, 2532, 1299,
       2094,  314, 4129, 2762, 2484, 3614, 4474, 3635, 4429, 2518,   67,
       3668, 1213, 2384, 4535, 6282, 6310, 5089, 1938, 1001, 6237, 1926,
       1672, 3635, 3134, 4005, 2954, 1687, 6046, 1561, 6073, 4667, 5513,
       4872, 2845, 6511, 3049, 2252, 1435, 3933, 3648, 2464, 2335, 4618,
       2964, 1274, 4279, 3517, 2762, 5885, 4560, 3542, 5472, 1460, 3229,
       2874, 4032, 2056, 1780, 4392, 5325, 3834, 6271, 3937, 4715, 1540,
       2795, 5056, 1200, 1671, 3649, 4400, 4551, 4807, 1794, 1007, 2849,
       3170, 3453, 3006, 5151, 6292, 3893, 1358, 6341, 3204, 3109, 2610,
        938])

In [38]:
print("Input: \n", repr(" ".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr(" ".join(idx2char[sampled_indices ])))

Input: 
 "pris feu   comment sort on de ia   j'y travaille travaille plus vite   ou vas tu   de i'autre cote de ce mur  prepare toi a fuir  quoi qu'il arrive indy  ne me laisse pas toute seule  c'est bon  tiens toi prete   indy  la torche s'eteint  regarde  regarde  ils envoient i'arche par avion le temps qu'ils la chargent  on sera a bord monsieur  buvons a notre succes dans le desert a i'arche  quand nous serons tres loin d'ici  et"

Next Char Predictions: 
 "entrepot accrocheur vue serpents excitants ci croyez soir qu'attendez frites couteau enfer arrivera nier hennessy forme m'engager peste m'interroger pension frappe admirer maitresse contraire ferai plaignants unanime vaincus reflechis douzaine cisco trous doublement demoli m'interroger joseph moyenne infraction depeche tente datte tetu pourrez salauds qu'allez i'histoire voyiez j'appartiendrai eustace d'assaut mois machoire folle faisait porte inscriptions coupes paierez lit hennessy supposez pleins longue s'imbriquent d'encaiss

# Train the model 

In [39]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 6553)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       8.78817


In [0]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [0]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 100 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [0]:
EPOCHS=100

In [0]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

# Generate text

In [0]:
tf.train.latest_checkpoint(checkpoint_dir)

In [0]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [0]:
model.summary()

In [0]:
#on enregistre le modèle : 
model.save('ModelGenerateText.h5')

#recreate the exact same model purely from the file
model = keras.models.load_model('ModelGenerateText.h5')

In [0]:
model